First let's try to read in the data and plot it with pyart

In [1]:
%matplotlib inline

In [2]:
import pyart
import numpy as np
import matplotlib.pyplot as plt
import yt


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
radar=pyart.io.read_nexrad_archive("./TJUA20170907_000014_V06")

In [4]:
import pyart.core.transforms

In [49]:
data = []
connectivity = []
coordinates = []
total = 0
cc = np.pi / 180.0
for sweep in range(6, 17):
    print(sweep)
    f = radar.get_field(sweep, "reflectivity")
    f = f.transpose()[:,:, None]
    data.append(f.copy("C"))
    r, theta, phi = radar.range['data'].copy(), radar.get_azimuth(sweep) * cc, radar.get_elevation(sweep) * cc
    r = pyart.core.transforms._interpolate_range_edges(r)
    theta = pyart.core.transforms._interpolate_azimuth_edges(theta)
    phi = pyart.core.transforms._interpolate_elevation_edges(phi)
    phi = np.array([phi.min(), phi.max()])
    
    print("Generating connectivity", r.shape, theta.shape, phi.shape)
    coords, conn = yt.hexahedral_connectivity(r, theta, phi)
    print("Generated")
    connectivity.append(conn + total)
    coordinates.append(coords)
    total += coords.shape[0]
connectivity = np.concatenate(connectivity)
coordinates = np.concatenate(coordinates, axis=0)
data = np.concatenate(data)
print(connectivity.shape, coordinates.shape, data.shape)

6
Generating connectivity (1833,) (361,) (2,)
Generated
7
Generating connectivity (1833,) (361,) (2,)
Generated
8
Generating connectivity (1833,) (361,) (2,)
Generated
9
Generating connectivity (1833,) (361,) (2,)
Generated
10
Generating connectivity (1833,) (361,) (2,)
Generated
11
Generating connectivity (1833,) (361,) (2,)
Generated
12
Generating connectivity (1833,) (361,) (2,)
Generated
13
Generating connectivity (1833,) (361,) (2,)
Generated
14
Generating connectivity (1833,) (361,) (2,)
Generated
15
Generating connectivity (1833,) (361,) (2,)
Generated
16
Generating connectivity (1833,) (361,) (2,)
Generated
(7254720, 8) (14557686, 3) (20152, 360, 1)


In [40]:
coordinates -= coordinates.min(axis=0)
coordinates /= coordinates.ptp(axis=0)

In [50]:
bbox = np.array([ coordinates.min(axis=0), coordinates.max(axis=0) ]).T
print(bbox)
ds = yt.load_hexahedral_mesh({'reflectivity': data}, connectivity, coordinates,
                            bbox = bbox, geometry = ("spherical"))

[[  2.00000000e+03   4.60000000e+05]
 [  1.74010946e-02   6.26748597e+00]
 [  3.02481819e-02   3.40567708e-01]]


In [51]:
yt.SlicePlot(ds, "phi", "reflectivity")

ValueError: attempt to get argmax of an empty sequence